In [1]:
from transformers import AutoProcessor, AutoModelForCausalLM, AutoModel, AutoTokenizer
from optimum.onnxruntime import ORTModelForSequenceClassification
from optimum.exporters import TasksManager
from PIL import Image
import numpy as np
import torch

2024-11-21 14:49:38.045268: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-21 14:49:38.211720: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732180778.299744   54638 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732180778.328239   54638 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-21 14:49:38.492957: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
model_id = "microsoft/Florence-2-base"
model = AutoModelForCausalLM.from_pretrained(
    model_id, trust_remote_code=True, torch_dtype="float32"
).eval()
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

/home/mnjm/.py_venv/clip/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [3]:
model

Florence2ForConditionalGeneration(
  (vision_tower): DaViT(
    (convs): ModuleList(
      (0): ConvEmbed(
        (proj): Conv2d(3, 128, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
        (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      )
      (1): ConvEmbed(
        (proj): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      )
      (2): ConvEmbed(
        (proj): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
      )
      (3): ConvEmbed(
        (proj): Conv2d(512, 1024, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      )
    )
    (blocks): ModuleList(
      (0): MySequential(
        (0): MySequential(
          (spatial_block): SpatialBlock(
            (conv1): PreNorm(
              (fn): Depth

In [4]:
processor

Florence2Processor:
- image_processor: CLIPImageProcessor {
  "auto_map": {
    "AutoProcessor": "microsoft/Florence-2-base--processing_florence2.Florence2Processor"
  },
  "crop_size": {
    "height": 768,
    "width": 768
  },
  "do_center_crop": false,
  "do_convert_rgb": null,
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_processor_type": "CLIPImageProcessor",
  "image_seq_length": 577,
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "processor_class": "Florence2Processor",
  "resample": 3,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 768,
    "width": 768
  }
}

- tokenizer: BartTokenizerFast(name_or_path='microsoft/Florence-2-base', vocab_size=50265, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '

In [5]:
onnx_path = "florence-2-base.onnx"
# task = "default"  # Adjust based on your model type
# onnx_config = TasksManager.get_exporter_config_for_model(model, task, exporter="onnx")

# model.export(
#     onnx_path=onnx_path,
#     config=onnx_config,
#     device="cpu"  # or "cuda" if GPU-based export is supported
# )
# print(f"Model exported to {onnx_path}")

In [6]:
dummy_image = Image.fromarray(np.zeros((224, 224, 3), dtype=np.uint8))
text = "An example input text for conversion"
inputs = processor(text=text, images=dummy_image, return_tensors="pt")

input_ids = inputs['input_ids']
attention_mask = inputs.get('attention_mask', torch.ones_like(input_ids, dtype=torch.long))

input_ids = inputs['input_ids']
pixel_values = inputs['pixel_values']
attention_mask = inputs.get('attention_mask', torch.ones_like(input_ids, dtype=torch.long))

attention_mask

tensor([[1, 1, 1, 1, 1, 1, 1, 1]])

In [7]:
output_path = "florence-2-base.onnx"
torch.onnx.export(
        model,
        (input_ids, pixel_values, attention_mask),
        output_path,
        export_params=True,
        opset_version=14,
        verbose=True,
        # input_names=[
        #     'input_ids', 
        #     'pixel_values', 
        #     'attention_mask'
        # ],
        # output_names=['outputs'],
        # dynamic_axes={
        #     'input_ids': {0: 'batch_size', 1: 'sequence'},
        #     'pixel_values': {0: 'batch_size', 1: 'channels', 2: 'height', 3: 'width'},
        #     'attention_mask': {0: 'batch_size', 1: 'sequence'},
        #     'outputs': {0: 'batch_size'}
        # }
    )

/home/mnjm/.cache/huggingface/modules/transformers_modules/microsoft/Florence-2-base/ceaf371f01ef66192264811b390bccad475a4f02/modeling_florence2.py:278: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert N == H * W
/home/mnjm/.cache/huggingface/modules/transformers_modules/microsoft/Florence-2-base/ceaf371f01ef66192264811b390bccad475a4f02/modeling_florence2.py:428: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert L == H * W, "input feature has wrong size"
/home/mnjm/.cache/huggingface/modules/transformers_modules/microsoft/Florence-2-base/ceaf371

ValueError: If no `decoder_input_ids` or `decoder_inputs_embeds` are passed, `input_ids` cannot be `None`. Please pass either `input_ids` or `decoder_input_ids` or `decoder_inputs_embeds`.